---
comments: true
layout: post
title: Setting up Cockpit
description: Setting up Cockpit
---

# Install Cockpit

In [ ]:
sudo apt update
sudo apt install cockpit

Start the Cockpit service

In [ ]:
sudo systemctl start cockpit

# Accessing

Cockpit runs on port 9090. This port needs to be allowed on the EC2 instance. Click on "Security Groups" in the left sidebar: 

![]({{ site.baseurl }}/images/securityGroups.jpg)

Next, search for your security group. Click the check box next to the group name, scroll down, and click on "Inbound rules". Click on "Edit inbound rules", and add a rule that allows "Port range" of 9090 and "Source type" of Anywhere-IPv4.

![]({{ site.baseurl }}/images/editRules.jpg)

<br>

Access Cockpit with the link: `<EC2 IP address>:9090`. There are a variety of ways to find the IP address of the instance. One way is to locate the IP address under "Public IPv4 address" on the "Instances" page:

![]({{ site.baseurl }}/images/ip.jpg)

In this case, the IP address of the instance is 3.145.69.15. Accessing http://3.145.69.15:9090 shows the Cockpit login screen:

![]({{ site.baseurl }}/images/cockpitHomepage.jpg)

<br>
Login with the same username and password used to login to the EC2 instance.

# Configure subdomain

1. Go to AWS Route 53 -> Hosted zone -> nighthawkcodingsociety.com

2. Click on "Create record"

3. Configure a subdomain, keep the "Record type" as "A", and add the IP address of the instance in the "Value" section.

<br>

Now you can access Cockpit at `<subdomain>:9090`.

# Nginx

Install Nginx to configure reverse proxy for Cockpit:

In [ ]:
sudo apt install nginx 

Create an nginx file in `/etc/nginx/sites-available/<your file name>` and add the following lines for Cockpit:

```
server {
    server_name    cockpit.nighthawkcodingsociety.com;

    location / {
        # Required to proxy the connection to Cockpit
        proxy_pass https://127.0.0.1:9090;
        proxy_set_header Host $host;
        proxy_set_header X-Forwarded-Proto $scheme;

        # Required for web sockets to function
        proxy_http_version 1.1;
        proxy_buffering off;
        proxy_set_header Upgrade $http_upgrade;
        proxy_set_header Connection "upgrade";

        # Pass ETag header from Cockpit to clients.
        # See: https://github.com/cockpit-project/cockpit/issues/5239
        gzip off;
    }
}
```

Create a certificate for the website. 

In [ ]:
# Update to latest version of snapd
sudo snap install core
sudo snap refresh core

# Install Certbot
sudo snap install --classic certbot

# Allow certbot to run
sudo ln -s /snap/bin/certbot /usr/bin/certbot

# Make Certbot edit Nginx configuration to enable HTTPS
sudo certbot --nginx

# Restart Nginx
sudo systemctl restart nginx

# To be continued

Things to research/add:

* Cockpit features